## Data Preparation

In [223]:
import pandas as pd
import numpy as np


In [224]:
# read the table from the html page and save it as type list
postal_code = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)
type(postal_code)


list

In [225]:
# the first element of the list is the required table. the type is now a dataframe
postal_code = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)[0]

type(postal_code)

pandas.core.frame.DataFrame

In [226]:
# assigning the correct columns name 
postal_code.columns = ["PostalCode", "Borough", "Neighborhood"]

In [227]:
postal_code.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [228]:
# condition for Borough with value "Not assigned"
mask = postal_code["Borough"] == "Not assigned"

In [229]:
# generating dataframe with Borough that are assigned
postal_codeC = postal_code[~mask]

In [230]:
postal_codeC.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [231]:
# checking that column Neighborhood has no element with value "Not assigned"
A = postal_codeC["Neighborhood"] == "Not assigned"
B = postal_codeC[A]
len(B)


0

In [234]:

postal_codeC.index = range(0, 103, 1)
postal_codeC.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [233]:
# dataframe dimension
postal_codeC.shape

(103, 3)

## Getting the latitude and logitude of the Neighborhood

### reading the csv file

In [238]:
# reading the geospatial_data and renaming column to be compatible with previous dataframe postal_codeC
geo = pd.read_csv("http://cocl.us/Geospatial_data")
geo.columns = ["PostalCode", "Latitude", "Longitude"]
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### combining the 2 dataframes to get a single dataframe with latitude and longitude included

In [245]:
#inner joint the 2 dataframe using the common column PostalCode
canada_code = pd.merge(left = postal_codeC,  right = geo, left_on = 'PostalCode', right_on = 'PostalCode')
canada_code.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
